In [11]:
from plotly import express as px

In [12]:
import plotly.graph_objects as go
import polars as pl
import torch

from cci.utils import project_dir

# res_dir = project_dir() / "results/CNN/32"
# res_dir = project_dir() / "results/CNN/40"
# res_dir = project_dir() / "results/CNN/47"
# res_dir = project_dir() / "results/CNN/65"
# res_dir = project_dir() / "results/CNN/101"
# res_dir = project_dir() / "results/CNN/114"
# Her++++++++++++++++++++++++++++++++++++++++++++++++4444
# res_dir = project_dir() / "results/CNN/115"4
# res_dir = project_dir() / "results/CNN/136"
# res_dir = project_dir() / "results/CNN/153"
# res_dir = project_dir() / "results/CNN/161"
# res_dir = project_dir() / "results/CNN/169"
# res_dir = project_dir() / "results/CNN/171"
# res_dir = project_dir() / "results/CNN/174"

res_dir = project_dir() / "results/CNN_vft/91"
res_dir = project_dir() / "results/CNN_full/66"
res_dir = project_dir() / "results/CNN_bigger_kernel_full_fixed/160"
res_dir = project_dir() / "results/CNN_bigger_kernel_vft_fixed/196"
res_dir = project_dir() / "results/CNN_bigger_kernel_as_fixed/151"
res_dir = project_dir() / "results/CNN_bigger_kernel_full_fixed/349"
res_dir = project_dir() / "results/CNN_even_bigger_kernel_mlp_full_fixed/160"
# model = torch.load(res_dir / "0_model.pt", map_location=torch.device("cpu"))
# print()

In [13]:
test_df = pl.read_csv(res_dir / "*_test.csv").drop("epoch")
test_df

acc,loss,f1,auroc,precision,recall,specificity,bac,TP,FP,FN,TN
f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64
0.775701,0.573737,0.868132,0.584483,0.831579,0.908046,0.2,0.554023,4,16,8,79
0.757009,1.076037,0.847059,0.610919,0.86747,0.827586,0.45,0.638793,9,11,15,72
0.738318,1.072425,0.835294,0.612069,0.855422,0.816092,0.4,0.608046,8,12,16,71
0.766355,1.00527,0.857143,0.606897,0.852273,0.862069,0.35,0.606034,7,13,12,75
0.775701,1.044443,0.860465,0.624138,0.870588,0.850575,0.45,0.650287,9,11,13,74


In [14]:
import numpy as np

from cci.metrics import confusion_matrix

# BUG: Fixed now but..
test_len = 107  # length of test set
for tp, fp, fn, tn in test_df.select(["TP", "FP", "FN", "TN"]).rows():
    tp = test_len - fp - fn - tn
    cm = np.array([[tp, fp], [fn, tn]]).astype("float")
    normalized_cm = cm / cm.sum(axis=1)[:, np.newaxis]
    fig = px.imshow(
        normalized_cm,
        labels=dict(x="Predicted", y="Actual"),
        x=["Good (0)", "Bad (1)"],
        y=["Good (0)", "Bad (1)"],
        text_auto=True,
    )
    fig.show()

In [15]:
def get_df(group) -> pl.DataFrame:
    df = pl.read_csv(res_dir / f"0_{group}.csv").with_columns(
        pl.lit(0).alias("fold"),
        pl.lit(group).alias("group"),
        pl.lit(f"{group}_0").alias("identifier"),
    )

    try:
        for i in range(1, 5):
            df = df.vstack(
                pl.read_csv(res_dir / f"{i}_{group}.csv").with_columns(
                    pl.lit(i).alias("fold"),
                    pl.lit(group).alias("group"),
                    pl.lit(f"{group}_{i}").alias("identifier"),
                )
            )
    except:
        pass

    return df


df = get_df("train")

df = df.vstack(get_df("val"))
df

epoch,acc,loss,f1,auroc,precision,recall,specificity,bac,TP,FP,FN,TN,fold,group,identifier
i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i32,str,str
1,0.419146,0.245099,0.525871,0.460711,0.805825,0.390282,0.555556,0.472919,75,60,389,249,0,"""train""","""train_0"""
2,0.447607,0.242054,0.553814,0.518716,0.830721,0.415361,0.6,0.50768,81,54,373,265,0,"""train""","""train_0"""
3,0.630013,0.242076,0.753448,0.539231,0.837165,0.684953,0.37037,0.527662,50,85,201,437,0,"""train""","""train_0"""
4,0.591203,0.237845,0.709559,0.59315,0.857778,0.605016,0.525926,0.565471,71,64,252,386,0,"""train""","""train_0"""
5,0.407503,0.23886,0.491111,0.585487,0.843511,0.346395,0.696296,0.521346,94,41,417,221,0,"""train""","""train_0"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
204,0.853081,0.430357,0.911175,0.850952,0.913793,0.908571,0.583333,0.745952,21,15,16,159,4,"""val""","""val_4"""
205,0.810427,0.482791,0.876543,0.896667,0.95302,0.811429,0.805556,0.808492,29,7,33,142,4,"""val""","""val_4"""
206,0.843602,0.356788,0.904348,0.888095,0.917647,0.891429,0.611111,0.75127,22,14,19,156,4,"""val""","""val_4"""


In [16]:
# fig = px.line(df, x="epoch", y="loss", line_group="identifier", color="group", height=800)


def avg_plot(metric: str):
    fig = go.Figure()
    for x, color in (["train", "blue"], ["val", "red"]):
        df = get_df(x)
        loss_df = (
            df.group_by("epoch", maintain_order=True)
            .agg(pl.col(metric).mean().alias("avg"), pl.col(metric).std().alias("std"))
            .to_pandas()
        )
        fig.add_trace(
            go.Scatter(
                name=f"Avg {x} {metric}",
                x=loss_df["epoch"],
                y=loss_df["avg"],
                mode="lines",
            )
        )
        fig.add_trace(
            go.Scatter(
                name=f"{x}upper bound",
                x=loss_df["epoch"],
                y=loss_df["avg"] + loss_df["std"],
                line=dict(width=0),
                marker=dict(color=color),
                showlegend=False,
            )
        )
        fig.add_trace(
            go.Scatter(
                name=f"{x}lower bound",
                x=loss_df["epoch"],
                y=loss_df["avg"] - loss_df["std"],
                line=dict(width=0),
                marker=dict(color=color),
                fill="tonexty",
                showlegend=False,
            ),
        )
    fig.update_layout(hovermode="x", height=800)
    fig.show()


avg_plot("loss")

In [17]:
avg_plot("bac")

In [18]:
avg_plot("acc")

In [19]:
avg_plot("f1")

In [20]:
avg_plot("auroc")